In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt

In [2]:
root = '../characters_parser/data/'
path = root + 'merged_grouple_data.csv'
df = pd.read_csv(path)
df.head()

,url_title,page_file,volumes_data_file
0,the_emperor_s_treasure,4425.json,NaN
1,katekyo_hitman_reborn__dj___small___round_kyou...,3782.json,NaN
2,buistvo_molodoi_dushi,1499.json,349.json
3,meaningful_kiss,5509.json,NaN
4,choice_of_love,507.json,431.json


In [3]:
print(df.shape)
print(df[~df.volumes_data_file.isna()].shape)

(8153, 3)
(1606, 3)


In [4]:
df = df[~df.volumes_data_file.isna()]
df.head()

,url_title,page_file,volumes_data_file
2,buistvo_molodoi_dushi,1499.json,349.json
4,choice_of_love,507.json,431.json
7,do_something_honey_for_me,1572.json,1396.json
12,moia_jizn,1393.json,939.json
15,first_girl,187.json,734.json


In [5]:
def load_data(page_file, volumes_file):
    page = root + 'grouple_manga_pages/' + page_file
    volumes = root + 'grouple_merged_volumes/' + volumes_file
    return {'page': json.load(open(page)),
            'volumes': json.load(open(volumes))}

# Fetch pairings

In [6]:
from pairing_utils import get_pairing

In [7]:
c = 0
for i, (name, page, volumes) in enumerate(df.to_records(index=False)):
    if c > 5:
        break
    
    data = load_data(page, volumes)
    url = data['page']['url']
    descr = data['page']['description'].replace('\n', ' ')
    is_pairing = 'пейринг' in descr.lower()

    if is_pairing:
        print(url, get_pairing(descr))
        c += 1

https://mintmanga.live/free__dj___awkward_sleepover ('Харука', 'Рин')
https://mintmanga.live/yowamushi_pedal_dj___juicy_hold_hole ('Тодо', 'Макисима')
https://mintmanga.live/katekyo_hitman_reborn__dj___a_game_for_new_suits ('Ямамото', 'Гокудера')
https://mintmanga.live/durarara___dj___biting_because_you_re_mine ('Shizuo', 'Izaya')
https://mintmanga.live/vse_vmeste ('Кенсей', 'Шуухей')
https://mintmanga.live/black_butler_dj___cheshire None


# Get most probable fandom

In [8]:
ydf1 = pd.read_csv(root + 'you_anime_characters_refs.csv', sep='\t')
ydf1['anime_url'] = ydf1.page_link.apply(lambda s: s.split('/')[-1])

ydf2 = pd.read_csv(root + 'you_anime_refs.csv', sep='\t')
ydf2['anime_url'] = ydf2.link.apply(lambda s: s.split('/')[-1])

ydf = pd.merge(ydf1, ydf2, on='anime_url')
ydf = ydf['title name'.split()]
ydf.head(3)

,title,name
0,Атака титанов,Эрен Йегер
1,Атака титанов,Микаса Аккерман
2,Атака титанов,Армин Арлерт


In [9]:
from pairing_utils import get_probable_fandoms

In [10]:
c = 0
for i, (name, page, volumes) in enumerate(df.to_records(index=False)):
    if c > 5:
        break
    data = load_data(page, volumes)
    url = data['page']['url']
    descr = data['page']['description'].replace('\n', ' ')
    is_pairing = 'пейринг' in descr.lower()

    if is_pairing:
        res = get_pairing(descr)
        c += 1
        if res is not None:
            p1, p2 = res
            fandoms = get_probable_fandoms(ydf, p1, p2)
            print(url, p1, p2, fandoms)
        

https://mintmanga.live/free__dj___awkward_sleepover Харука Рин ['Вольный стиль! Спецвыпуски', 'Маленькие проказники: Я защищу мир Сайто!', 'Вольный стиль! Вечное соревнование — Обещание', 'Импульс мира', 'Актёры ослеплённого города', 'Маленькие проказники! Ещё раз', 'Вольный стиль! Займите свои позиции', 'Вольный стиль! Вечное лето — Спецвыпуск', 'Вперёд, ко вселенной!!', 'Дни Мидори', 'Вольный стиль!', 'Маленькие проказники! EX', 'Механизированные спасатели, вперёд!', 'Вольный стиль! Вечное лето', 'Вольный стиль! Вечное соревнование — Узы', 'Высокая скорость: Так начинался «Вольный стиль!»', 'Вольный стиль! Заплыв в будущее', 'Активный рейд: Мобильная боевая дивизия, восьмой отряд', 'Я вовсе не люблю своего старшего брата!', 'Идолмастер: За гранью светлого будущего!', 'Активный рейд: Мобильная боевая дивизия, восьмой отряд 2', 'Маленькие проказники!']
https://mintmanga.live/yowamushi_pedal_dj___juicy_hold_hole Тодо Макисима []
https://mintmanga.live/katekyo_hitman_reborn__dj___a_game_

# Group fandoms

In [11]:
# Examples
e0 = ['Актёры ослеплённого города', 'Активный рейд: Мобильная боевая дивизия, восьмой отряд 2', 'Вольный стиль! Займите свои позиции', 'Вольный стиль! Вечное лето', 'Маленькие проказники! Ещё раз', 'Высокая скорость: Так начинался «Вольный стиль!»', 'Маленькие проказники: Я защищу мир Сайто!', 'Вольный стиль! Вечное соревнование — Узы', 'Я вовсе не люблю своего старшего брата!', 'Вольный стиль! Вечное соревнование — Обещание', 'Импульс мира', 'Вольный стиль! Вечное лето — Спецвыпуск', 'Активный рейд: Мобильная боевая дивизия, восьмой отряд', 'Вольный стиль! Заплыв в будущее', 'Маленькие проказники! EX', 'Вперёд, ко вселенной!!', 'Вольный стиль!', 'Вольный стиль! Спецвыпуски', 'Маленькие проказники!', 'Дни Мидори', 'Механизированные спасатели, вперёд!', 'Идолмастер: За гранью светлого будущего!']
e1 =  ['Гинтама: Джамп Феста 2005', 'Гинтама: Лучшее из «Гинтамы» на Theater 2D', 'Гинтама 8', 'Гинтама x Фасоль', 'Гинтама', 'Гинтама: Джамп Феста 2015', 'Гинтама 3', 'Гинтама: Финальная арка — Ёродзуя навсегда!', 'Гинтама: Сказание о Бэнидзакуре', 'Гинтама: Джамп Феста 2014', 'Гинтама: Грандиозное собрание', 'Гинтама 2', 'Гинтама: Рождение Белого Демона', 'Гинтама 7', 'Гинтама 4', 'Гинтама 6']
e2 = ['Ван-Пис: Запасной план. Идеальный метод захвата Ван-Пис', 'Ван-Пис: Золото', 'Ван-Пис: Театр Мугивары', 'Ван-Пис: Путешествие в один конец', 'Ван-Пис: Танцевальный марафон Джанго', 'Ван-Пис', 'Ван-Пис: Воспоминания детектива в соломенной шляпе', 'Ван-Пис: Эпизод Чоппера Плюс — Зимнее цветение, чудесная сакура', 'Ван-Пис 3D2Y: Пережить смерть Эйса! Обещание Луффи своим накама', 'Ван-Пис: Рекап', 'Ван-Пис: Эпизод Луффи — Приключения на Ладоневом острове', 'Ван-Пис: Приключение на Заводном Острове', 'Ван-Пис: Приключение в глуби океана', 'Ван-Пис: Z', 'Ван-Пис: Эпизод Сабо — Узы трёх братьев, чудесное воссоединение и унаследованная воля', 'Ван-Пис: Барон Омацури и Секретный Остров', 'Ван-Пис: Эпизод Нами — Слёзы навигатора и узы друзей', 'Ван-Пис: Футбольный король мечты', 'Ван-Пис 3D: Погоня за соломенной шляпой', 'Ван-Пис: Гигантский механический солдат замка Каракури', 'Ван-Пис: Проклятие Святого Меча', 'Ван-Пис: Защитить! Последнее великое представление!', 'Ван-Пис: Открытие в большом море! Большая мечта отца!', 'Ван-Пис: Остров мечты', 'Ван-Пис: Жестокий мир', 'Ван-Пис: Начало приключений', 'Ван-Пис: Приключение в Алабасте — Принцесса пустыни и пираты', 'Ван-Пис: Эпизод Ист Блю — Большое приключение Луффи и его команды', 'Ван-Пис: Приключения в Алабасте — Пролог', 'Ван-Пис: Золото — Эпизод 0', 'Ван-Пис: Королевство Чоппера на Острове Необычных Животных', 'Ван-Пис: Приключение в Небуландии', 'Ван-Пис: Эпизод Мерри — История об ещё одном накаме', 'Ван-Пис: Цельсь! Пиратские короли бейсбола', 'Серена x Ван-Пис 3D: Погоня за Соломенной шляпой — Проникновение! Таузенд Санни!', 'Ван-Пис: Золотое сердце']
e3 = ['Синий экзорцист: Путешествие Куро', 'Новый принц тенниса OVA', 'Синий экзорцист', 'Новый принц тенниса', 'Синий экзорцист: Спецвыпуски', 'Новый принц тенниса: Спецвыпуски', 'Синий экзорцист: Нечестивый король Киото', 'Блич', 'Синий экзорцист: Нечестивый король Киото OVA']
e4 =  ['Атака титанов: Рекап', 'Атака титанов OVA', 'Атака титанов 3', 'Атака титанов: Выбор без сожалений', 'Атака титанов: Крылья свободы', 'Атака титанов: Багровые луки и стрелы', 'Атака титанов 2', 'Атака титанов: Театр с монстрами — Вперёд, кадетский корпус!', 'Атака! Титаны средней школы', 'Атака титанов']
e5 = ['Маги: Королевство магии', 'Маги: Лабиринт магии']

In [12]:
from pairing_utils import group_fandoms

In [13]:
all_cases = (e0,e1,e2,e3,e4,e5)
one_case = (e5, )
for e in all_cases:
    for key, val in group_fandoms(e).items():
        print(key)
        for v in val:
            print('  ', v)
            
    print('--------------------')

Вольный стиль!
   Вольный стиль!
   Вольный стиль! Вечное лето
   Вольный стиль! Спецвыпуски
   Вольный стиль! Заплыв в будущее
   Вольный стиль! Займите свои позиции
   Вольный стиль! Вечное лето — Спецвыпуск
   Вольный стиль! Вечное соревнование — Узы
   Вольный стиль! Вечное соревнование — Обещание
Маленькие проказники!
   Маленькие проказники!
   Маленькие проказники! EX
   Маленькие проказники! Ещё раз
   Маленькие проказники: Я защищу мир Сайто!
Активный рейд: Мобильная боевая дивизия, восьмой отряд
   Активный рейд: Мобильная боевая дивизия, восьмой отряд
   Активный рейд: Мобильная боевая дивизия, восьмой отряд 2
Дни Мидори
   Дни Мидори
Импульс мира
   Импульс мира
Вперёд, ко вселенной!!
   Вперёд, ко вселенной!!
Актёры ослеплённого города
   Актёры ослеплённого города
Механизированные спасатели, вперёд!
   Механизированные спасатели, вперёд!
Я вовсе не люблю своего старшего брата!
   Я вовсе не люблю своего старшего брата!
Идолмастер: За гранью светлого будущего!
   Идолмасте

In [14]:
n_pairings = 0
n_recognized = 0

for i, (name, page, volumes) in enumerate(df.to_records(index=False)):
    data = load_data(page, volumes)
    url = data['page']['url']
    descr = data['page']['description']
    is_pairing = 'пейринг' in descr.lower()

    if is_pairing:
        n_pairings += 1
        res = get_pairing(descr)
        if res is not None:
            p1, p2 = res
            fandoms = get_probable_fandoms(ydf, p1, p2)
            groupped_fandoms = list(group_fandoms(fandoms).keys())
            if len(groupped_fandoms):
                n_recognized += 1
                print(p1, '+', p2)
                print('  ', groupped_fandoms, url)
        

Харука + Рин
   ['Вольный стиль!', 'Маленькие проказники!', 'Активный рейд: Мобильная боевая дивизия, восьмой отряд', 'Дни Мидори', 'Импульс мира', 'Вперёд, ко вселенной!!', 'Актёры ослеплённого города', 'Механизированные спасатели, вперёд!', 'Я вовсе не люблю своего старшего брата!', 'Идолмастер: За гранью светлого будущего!', 'Высокая скорость: Так начинался «Вольный стиль!»'] https://mintmanga.live/free__dj___awkward_sleepover
Юкио + Рин
   ['Синий экзорцист', 'Новый принц тенниса', 'Блич'] https://mintmanga.live/dogra
Эрвин + Леви
   ['Атака титанов', 'Атака! Титаны средней школы'] https://mintmanga.live/eche_ne_vremia
Шисуи + Итачи
   ['Наруто: Ураганные хроники'] https://mintmanga.live/naruto_dj___enjoy_
Гинтоки + Хиджиката
   ['Гинтама'] https://mintmanga.live/spazm
Зоро + Санджи
   ['Ван-Пис', 'Серена x Ван-Пис 3D: Погоня за Соломенной шляпой — Проникновение! Таузенд Санни!'] https://mintmanga.live/one_piece_dj___kaisui_pearl
Масрур + Синдбад
   ['Маги:'] https://mintmanga.live

KeyboardInterrupt: 

In [ ]:
print('Описаний с пейрингами:', n_pairings)
print('Распознано фандомов по пейрингам:', n_recognized, '(%.2f' % (n_recognized/n_pairings*100) + '%)')

class Manga# Create NER dataset